# Libraries

In [1]:
import numpy as np
import pandas as pd
import mne
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from tqdm import tqdm
from bids import BIDSLayout
from bids.tests import get_test_data_path
from joblib import Parallel, delayed
from numba import jit, cuda

# Functions

## information rate adjacent pdf

In [2]:
# @title Information rate [KDE] adjacent --> info_rate_kernel_adj()
#[information rate] calculation based on the [adjacent PDF] (PDF random variable limits are based on TWO adjacent window) instead of having the PDF LIMITED to global [max] and [min]
def info_rate_kernel_adj(signal, time, window, slide, pdf_point=10000):
    assert signal.shape[0] > 10, 'signal: data of the given signal needs to be sufficient. Increase the number of data.'
    assert time.shape[0] == signal.shape[0], 'time: the length of the time needs to be the same as the signal. Check the given time.'
    assert isinstance(pdf_point, (int)), 'pdf_point: # points of a PDF, integer is needed.'
    assert isinstance(window, (int)), 'window: # sample size to form PDF, integer is needed.'
    assert isinstance(slide, (int)), 'slide: # shifting points to form the adjacent PDF, integer is needed.'

    temp_data = signal
    temp_time = time

    win = window
    sld = slide
    sample = pdf_point
    factor = 0.01 #+- (factor)*100% away from the data

    all_ratedata = []
    ratetime = []

    #########################
    all_pdf = []
    all_axs = []
    #########################

    for t in tqdm(range(0, len(temp_time)-win, sld), position=0):
        dmin = np.min(temp_data[t:t+win+sld])
        dmax = np.max(temp_data[t:t+win+sld])
        d_buffer = factor * (dmax - dmin)
        dmin -= d_buffer
        dmax += d_buffer

        #[gaussian] kernel is used to get a smooth PDF so that it could recover the "actual change" for small sliding window
        #[silverman] bandwidth is used as it is the [standard] in MATLAB and quick calculation; it would be more accurate to use [ISJ] instead but need more time.
        temp_axs = np.linspace(dmin, dmax, sample) #the range of the PDF.
        pdf1 = FFTKDE(kernel='box', bw='silverman').fit(temp_data[t:t+win]).evaluate(temp_axs) #1st PDF
        pdf2 = FFTKDE(kernel='box', bw='silverman').fit(temp_data[t+sld:t+win+sld]).evaluate(temp_axs) #2nd PDF

        pdf1[pdf1 < 0] = 0 #if any values less than 0, it will be ZERO
        pdf2[pdf2 < 0] = 0 #if any values less than 0, it will be ZERO

        #########################
        all_pdf.append([pdf1, pdf2])
        all_axs.append([temp_axs, temp_axs])
        #########################

        temp_inforate= 4*np.sum(((pdf2)**(1/2) - (pdf1)**(1/2))**(2)) * ((temp_axs[1]-temp_axs[0])/((temp_time[t]-temp_time[t+sld])**(2))) #information rate calculation

        all_ratedata.append(temp_inforate)
        ratetime.append(temp_time[t])

    all_ratedata = np.array(all_ratedata)
    ratetime = np.array(ratetime)

    #########################
    all_pdf = np.array(all_pdf)
    all_axs = np.array(all_axs)
    #########################

    print('[information rate amplitude] DONE!')
    #print('\n', 'return [info_rate], [info_rate time], [all_pdf], [pdf axes]')

    return all_ratedata, ratetime, all_pdf, all_axs

## window sliding for 3 dimensions [T x N x 1]

In [3]:
# @title Sliding window for [leading eigenvector] --> win_sld().... Use this as reference for [broadcasting].
def win_sld(eigvec_data, window=1000, slide=500):
    assert isinstance(eigvec_data, (np.ndarray)), 'eigvec_data: [leading eigenvector] should be in numpy.array format.'
    assert len(eigvec_data.shape) == 3, 'eigvec_data: [leading eigenvector] should be in 3 dimension as [T x N x 1].'
    assert isinstance(window, (int)), 'window: sampling size (window size) should be [Integer]; default is 1000 points.'
    assert isinstance(slide, (int)), 'slide: [sliding of window] should be [Integer]; default is 500 points.'

    window_data = eigvec_data

    temp_win = window
    temp_sld = slide

    temp_number_window = (window_data.shape[0] - temp_win) // temp_sld + 1
    temp_result_shape = (temp_number_window, temp_win * window_data.shape[1], window_data.shape[2])

    temp_indices = np.arange(temp_win) + np.arange(temp_number_window)[:, None] * temp_sld

    up_window_data = window_data[temp_indices].reshape(temp_result_shape)

    return up_window_data

## PDF estimation via histogram

In [4]:
# @title PDF estimated via histogram --> hist_est()
#The PDF is estimated via histogram. To speed up the calculation, numpy.apply_along_axis() is used.
from timeit import default_timer as timer

def hist_est(window_data, bins_size=10, int_range=(-1.1, 1.1)):
    assert isinstance(window_data, (np.ndarray)), 'window_data: the data need to be in numpy.array format.'
    assert len(window_data.shape) == 3, 'window_data: the array should have 3 dimensions as [T x N x 1].'
    assert isinstance(bins_size, (int)), 'bins_size: bins size of the histogram; default is 10. Integer is needed.'
    assert isinstance(int_range, (tuple)), 'int_range: range of interested axis; default is [-1.1 to 1.1]. Tuple is needed.'
    assert len(int_range) == 2, 'int_range: ONLY 2 values(min and max of axis) are needed.'

    temp_data_his = window_data
    temp_bins = bins_size
    temp_axs = np.linspace(np.min(int_range), np.max(int_range), temp_bins)

    start = timer() #timer start to estimate calculation time.
    temp_his = np.apply_along_axis(lambda temp_data_his: np.histogram(temp_data_his, temp_bins, range=[np.min(temp_axs), np.max(temp_axs)], density=True)[0], axis=1, arr=temp_data_his)
    print(f'time taken: {timer() - start}') #print the time for the whole process.
    print(f'PDF evolution estimatation via histogram done!!!')

    #return the [histogram data] and [axis of histogram].
    return temp_his, temp_axs

## phase lock matrix

In [5]:
# @title Phase lock matrix --> phase_lock_matrix()
#[phase lock matrix] calculation using the [hilbert analytic signal]
def phase_lock_matrix(hilbert_data):
    assert isinstance(hilbert_data, (mne.io.array.array.RawArray)), 'hilbert_data: data should be in MNE format of [hilbert analytic signal] for further calculation.'

    hilbert_angle = hilbert_data

    matrix_hilbert_angle = []
    for i in (hilbert_angle.ch_names):
        for j in hilbert_angle.ch_names:
            temp_dif_angle = hilbert_angle[i][0][0] - hilbert_angle[j][0][0]
            temp_dif_angle = np.cos(temp_dif_angle)
            matrix_hilbert_angle.append(temp_dif_angle)

    matrix_hilbert_angle = np.array(matrix_hilbert_angle)

    #reshape the matrix of the [phase-locking patterns] to have the [N x N x T] 3D matrix --> matrix at each time points
    matrix_hilbert_angle = np.reshape(matrix_hilbert_angle, (len(hilbert_angle.ch_names), len(hilbert_angle.ch_names), -1))

    #return the values of the [phase lock matrix] along with the [electrode channels]
    return matrix_hilbert_angle, hilbert_angle.ch_names

## window sliding correlation --> try to make it faster

In [6]:
# @title Correlation matrix --> corr_matrix()
#[sliding window correlation matrix] calculation
from scipy.stats import pearsonr

def corr_matrix(signal_data, window=1000, slide=500):
    assert isinstance(signal_data, (mne.io.array.array.RawArray)), 'signal_data: data should be in MNE format of [original signals] for further calculation.'
    assert isinstance(window, (int)), 'window: sampling data (window size). Integer is needed.'
    assert isinstance(slide, (int)), 'slide: sliding of sampling data. Integer is needed.'

    temp_data = signal_data
    temp_win = window
    temp_sld = slide

    matrix_correlation = []
    for i in (temp_data.ch_names):
        for j in temp_data.ch_names:
            #window sliding the signals
            temp_chnl1 = np.lib.stride_tricks.sliding_window_view(temp_data[i][0][0], window_shape=temp_win)[::temp_sld]
            temp_chnl2 = np.lib.stride_tricks.sliding_window_view(temp_data[j][0][0], window_shape=temp_win)[::temp_sld]



            #calculate the [correlation]
            for t in range(temp_chnl1.shape[0]):
                temp_corr, temp_pval = pearsonr(temp_chnl1[t,:], temp_chnl2[t,:])
                '''
                if temp_pval > 0.001:
                    print(t, temp_pval)
                    break
                '''
                matrix_correlation.append(temp_corr)

    matrix_correlation = np.array(matrix_correlation)

    #reshape the matrix of [correlation] to have [N x N x T] 3D matrix --> matrix at each time points
    matrix_correlation = np.reshape(matrix_correlation, (len(temp_data.ch_names), len(temp_data.ch_names), temp_chnl1.shape[0]))

    #return the values of the [correlation matrix] along with the [electrode channels]
    return matrix_correlation, temp_data.ch_names

## leading eigenvector

In [7]:
# @title TRYING JOBLIB Leading eigenvector --> lead_eigvec_cal()
#compute the [leading eigenvector] using scipy
#the [leading eigenvector] is computed at each time step
from scipy.linalg import eigh
from joblib import Parallel, delayed

def job_lead_eigvec_cal(matrix, i):
    temp_matrix = matrix[:, :, i]
    temp_eigval, temp_eigvec = eigh(temp_matrix)

    #sum_temp_eigval = np.sum(temp_eigval)
    #high_temp_eigval = np.max(np.abs(temp_eigval))
    loc_high_temp_eigval = np.argmax(np.abs(temp_eigval))
    #loc_high_temp_eigval = np.where(temp_eigval == high_temp_eigval)

    #the [reshape] is done to get the [N x 1] vector of the [leading eigenvector] else it will be [N] vector.
    #the [real] is needed to make sure to "get rid" of the [imaginary part], JUST in case. Can be removed since the eigenvalues is positive.

    #real_temp_eigvec = np.real(temp_eigvec[:, loc_high_temp_eigval])
    #high_temp_eigvec = (real_temp_eigvec / np.max(np.abs(real_temp_eigvec))).reshape(-1, 1)
    high_temp_eigvec = np.real(temp_eigvec[:, loc_high_temp_eigval]/np.max(np.abs(np.real(temp_eigvec[:, loc_high_temp_eigval])))).reshape(-1, 1)

    #The largest component is set to negative.
    #This step is needed because the same eigenvector can have a negative symmetric result.
    #To make sure the symmetric is always the same. Negative is chosen as the reference.
    #refer to [https://github.com/juanitacabral/LEiDA/blob/master/LEiDA/LEiDA.m]
    '''
    if np.mean(high_temp_eigvec[high_temp_eigvec > 0]) > 0.5:
        high_temp_eigvec = -1*high_temp_eigvec
    '''

    if len(high_temp_eigvec[high_temp_eigvec>0]) == 0:
        high_temp_eigvec = high_temp_eigvec
    elif np.mean(high_temp_eigvec[high_temp_eigvec>0]) > 0.5:
        high_temp_eigvec = -1*high_temp_eigvec
    elif np.mean(high_temp_eigvec[high_temp_eigvec > 0]==0.5) and np.sum(high_temp_eigvec[high_temp_eigvec>0]) > -1*np.sum(high_temp_eigvec[high_temp_eigvec>0]):
        high_temp_eigvec = -1*high_temp_eigvec

    '''
    else:
        high_temp_eigvec = high_temp_eigvec
    '''

    return high_temp_eigvec #, high_temp_eigval, sum_temp_eigval

def lead_eigvec_cal(matrix_data):
    assert isinstance(matrix_data, (np.ndarray)), 'matrix_data: the matrix needs to be in numpy.array format.'
    assert len(matrix_data.shape) == 3, 'matrix_data: the matrix needs to be [3 dimensions] such that it has dimension for [N x N x T].'
    assert matrix_data.shape[0] == matrix_data.shape[1], 'matrix_data: the '

    matrix_hilbert_angle = matrix_data

    start = timer()
    high_eigvec = Parallel(n_jobs=-1)(delayed(job_lead_eigvec_cal)(matrix_hilbert_angle, i) for i in (range(0, matrix_hilbert_angle.shape[2], 1)))
    print(f'time taken for leading eigenvector: {timer() - start}')

    high_eigvec = np.array(high_eigvec) #[leading eigenvector]

    '''
    high_eigval = np.array(high_eigval)
    sum_eigval = np.array(sum_eigval)
    prop_eigval = np.real(high_eigval)/np.real(sum_eigval) #calculate the proportional of [leading eigenvector] with respect to the [sum of eigenvector]
    '''

    #saving the [leading eigenvector] with [channels information] for future reference.
    #high_eigvec_data = mne.io.RawArray(high_eigvec[:,:,0].T, hilbert_angle.info)

    #return the [leading eigenvector] along with the [electrode channels].
    return high_eigvec

## K-means clustering

In [8]:
# @title K-means clustering --> kmeans_cal()
#compute the [kmeans clustering] via [sklearn].
from sklearn.cluster import KMeans

def kmeans_cal(vector_data, num_clus=5):
    assert isinstance(vector_data, (np.ndarray)), 'vector_data: vector needs to be numpy.ndarray.'
    assert len(vector_data.shape) == 3, 'vector_data: vector needs to be in the dimension of [T x N x 1].'
    assert isinstance(num_clus, (int)), 'num_clus: number of clusters; default is 5. Integer is needed.'

    high_eigvec = vector_data #leading eigenvector.
    num_cluster = num_clus #number of clusters.

    #kmeans clustering
    kmeans_data = KMeans(n_clusters=num_cluster, random_state=0, ).fit(high_eigvec[:,:,0])
    temp_label = kmeans_data.labels_ #location of different clusters.

    #locate the [time] for each [class]
    class_loc = []
    for i in range(0, np.max(temp_label)+1, 1):
        temp_class_loc = np.where(temp_label == i)[0]
        class_loc.append(temp_class_loc)

    #find out the [leading eigenvector] for each [class]
    temp_vector_time = high_eigvec
    class_data = []
    for i in range(0, len(class_loc), 1):
        temp_class_data = temp_vector_time[class_loc[i], :, :]
        class_data.append(temp_class_data)

    #calculate the mean and standard deviation for each [class]
    class_mean = []
    class_std = []
    for i in range(0, len(class_data), 1):
        temp_class_mean = np.mean(class_data[i], axis=0)
        temp_class_std = np.std(class_data[i], axis=0)

        class_mean.append(temp_class_mean)
        class_std.append(temp_class_std)
    class_mean = np.array(class_mean)
    class_std = np.array(class_std)

    #return the [mean of the class] and [standard deviation of the class] and [location for different clusters].
    return class_mean, class_std, temp_label

## Network mapping

In [9]:
# @title Graph network plot --> connect_network()
#compute the graph of the network using [networkx]
#NOTE: interative graph can be obtained via [pyvis] but it is quite tedious to display the plot.
#NOTE: interative graph can alternatively obtained via [plotly] but more setup is needed.
import networkx as nx

def connect_network(vector_data, chnl_data, show_condi=0.2):
    assert isinstance(vector_data, (np.ndarray)), 'vector_data: [lead eigenvector] which should have the dimension of [N x 1].'
    assert vector_data.shape[1] == 1, 'vector_data: [lead eigenvector] should have the [2nd dimension] equal to 1 as it is [N x 1] dimension.'
    assert isinstance(chnl_data, (list)), 'chnl_data: list of the [electrodes]. List is needed.'
    assert isinstance(show_condi, (float, int)), 'show_condi: condition value to allow the connection happens. Float is needed.'

    G = nx.Graph()
    #add the [nodes] to the [graph]
    for c in hilbert_angle.ch_names:
        G.add_node(c)

    #setting of the [nodes]
    options = {
        "font_size": 10,
        "node_size": 500,
        "node_color": "white",
        "edgecolors": "black",
        "linewidths": 1,
        "width": 1,
    }

    #setting the [position] of the [nodes]
    pos = {'Cz': (0,0),
           'C4': (0.5,0), 'T4': (1.1,0), 'A2': (1.5,0),
           'C3': (-0.5,0), 'T3': (-1.1,0), 'A1': (-1.5,0),
           'Fz': (0,0.5), 'Pz': (0,-0.5),
           'F4': (0.5,0.5), 'F8': (1,0.5),
           'F3': (-0.5,0.5), 'F7': (-1,0.5),
           'Fp1': (-0.5,1.0), 'Fp2': (0.5,1.0),
           'P4': (0.5,-0.5), 'T6': (1,-0.5),
           'P3': (-0.5,-0.5), 'T5': (-1,-0.5),
           'O2': (0.5,-1), 'O1': (-0.5,-1)}

    #information of the [nodes] and [edges] connection.
    temp_chnl = chnl_data #list of the [electrode channels]
    class_mean = vector_data #leading

    for i in class_mean:
        for j in class_mean[class_mean != i]:
            #increased the condition for the [lead eigenvector] to "truncate" the connectivity of the nodes
            if i>show_condi and j>show_condi:
                temp_loci = np.where(class_mean == i)[0][0]
                temp_locj = np.where(class_mean == j)[0][0]
                G.add_edge(temp_chnl[temp_loci], temp_chnl[temp_locj])

    nx.draw_networkx(G, pos, **options)
    ax = plt.gca()
    ax.margins(0.20)
    plt.axis('off')
    plt.show()

    return G.edges()

## Principal Component Analysis

In [10]:
# @title Principal Component Analysis
#This code uses [sklearn] to compute the [Principal Component Analysis].
#User can choose to have [specific components] or [interested information remain] (remember to set number_component = 0).
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def pca_function(dataset, tolerance=0.85, number_component=3):
    assert isinstance(dataset, (np.ndarray)), 'dataset: given the data need to be in [numpy.ndarray]. Convert it to [numpy.ndarray] from other type like [pandas.Dataframe].'
    assert isinstance(tolerance, (float)), 'tolerance: PCA retained information. Float number is needed. NOTE: 1 == 100% which should NOT be used here. Default is 0.85.'
    assert (tolerance > 0) and (tolerance < 1), 'tolerance: PCA retained information should be in the range of (0, 1). Default is 0.85.'
    #[number_component] == 0 --> process to [while loop] which determined the optimum [components] needed.
    assert isinstance(number_component, (int)), 'number_component: number of components for the PCA. Integer is needed. Default is 3.'

    new_data = dataset
    new_data = StandardScaler().fit_transform(new_data)

    temp_comp = 1
    temp_contribution = 0

    if number_component == 0:
        while temp_contribution <= tolerance:
            temp_pca_eeg = PCA(n_components=temp_comp)
            temp_principal_eeg = temp_pca_eeg.fit_transform(new_data)
            temp_contribution = np.round(np.sum(temp_pca_eeg.explained_variance_ratio_), 4)
            temp_comp = temp_comp + 1

    if number_component > 0:
        temp_pca_eeg = PCA(n_components=number_component)
        temp_principal_eeg = temp_pca_eeg.fit_transform(new_data)
        temp_contribution = np.round(np.sum(temp_pca_eeg.explained_variance_ratio_), 4)

    #print('Explained variation per principal component: {}'.format(temp_pca_eeg.explained_variance_ratio_))

    return temp_principal_eeg, temp_pca_eeg

## Cosine similarity classification

In [11]:
# @title cosine similarity classification [calculation] --> class_cos_time()
#[cosine similarity] classification --> classify the time where [vector of elements] are within the "tolerance".
def class_cos_time(norm_matrix, time, tolerance=0.9):
    assert norm_matrix.shape[0]>=1 and norm_matrix.shape[1]>=1, 'norm_matrix: given data matrix need to contain element for evaluation.'
    assert norm_matrix.all() <= 1, 'norm_matrix: element(s) must be normalized by MAXIMUM value. NO element should be more than 1.'
    assert time.shape[0] == norm_matrix.shape[0], 'time: the size of the time should be the same as [norm_matrix] time index.'
    assert isinstance(tolerance, (float, int)), 'tolerance: range of the accepted range must be either float or integer.'
    assert 0<= tolerance <= 1, 'tolerance: range of the accepted value is 0 to 1 for [cosine similarity].'

    ref_allelement = norm_matrix[0] #reference vector for [cosine similarity]
    ref_time = time
    all_cos_sim = []

    all_class_time = []
    all_notclass_time = []

    loc_class_time = []
    loc_notclass_time = []
    for t in tqdm(range(0, len(ref_time), 1), position=0):
        temp_cos_sim = cosine_similarity(ref_allelement.reshape(1, -1), norm_matrix[t].reshape(1, -1))[0][0]
        ########################################################################
        temp_euc_dis = euclidean_distances(ref_allelement.reshape(1, -1), norm_matrix[t].reshape(1, -1))[0][0]
        #print(f'euc_dist: {temp_euc_dis}, t: {t}')
        #print(f'cos_sim: {temp_cos_sim}, t: {t}')
        ########################################################################
        #all_cos_sim.append(temp_cos_sim) #calculate value of [cosine similarity]

        #store the time if the [cosine similarity] is within the [interested range]
        if (temp_cos_sim) >= tolerance:
            all_class_time.append(ref_time[t])
            loc_class_time.append(t)
        else:
            all_notclass_time.append(ref_time[t])
            loc_notclass_time.append(t)
    #all_cos_sim = np.array(all_cos_sim)
    all_class_time = np.array(all_class_time)
    all_notclass_time = np.array(all_notclass_time)
    loc_class_time = np.array(loc_class_time)
    loc_notclass_time = np.array(loc_notclass_time)

    #print('[classified time] and [not classified time]')

    return all_class_time, loc_class_time, all_notclass_time, loc_notclass_time

# Analysis

In [12]:
# @title Extract the data
#extract the data for [difference group of participants]
participants_file = '/home/hengjie/Desktop/alzheimer_openneuro/participants.tsv'

participants_df = pd.read_csv(participants_file, delimiter='\t')

#alzheimer group
alz_part = participants_df[participants_df['Group'] == 'A']
alz_id = alz_part['participant_id'].tolist()
alz_id = [s.replace('sub-', '') for s in alz_id]

#frontotemporal dementia group
dem_part = participants_df[participants_df['Group'] == 'F']
dem_id = dem_part['participant_id'].tolist()
dem_id = [s.replace('sub-', '') for s in dem_id]

#healthy group
con_part = participants_df[participants_df['Group'] == 'C']
con_id = con_part['participant_id'].tolist()
con_id = [s.replace('sub-', '') for s in con_id]

#particiapants data
print(participants_df)

   participant_id Gender  Age Group  MMSE
0         sub-001      F   57     A    16
1         sub-002      F   78     A    22
2         sub-003      M   70     A    14
3         sub-004      F   67     A    20
4         sub-005      M   70     A    22
..            ...    ...  ...   ...   ...
83        sub-084      F   71     F    24
84        sub-085      M   64     F    26
85        sub-086      M   49     F    26
86        sub-087      M   73     F    24
87        sub-088      M   55     F    24

[88 rows x 5 columns]


In [13]:
# @title Read the BIDS format dataset
#read the BIDS format dataset
#NOTE: the [derivatives] should use the [derivatives] way of reading it but it does not work in this case. Hence, the author copy the [metadata] to the [derivatives].
data_path = os.path.join(get_test_data_path(), '/home/hengjie/Desktop/alzheimer_openneuro/derivatives')
layout = BIDSLayout(data_path)

In [14]:
alz_eeg_file = layout.get(subject=alz_id, return_type='filename', extension='.set') #alzheirmer 
dem_eeg_file = layout.get(subject=dem_id, return_type='filename', extension='.set') #dementia
con_eeg_file = layout.get(subject=con_id, return_type='filename', extension='.set') #control/healthy

In [15]:
print(f'number of dataset: {len(dem_eeg_file)}')

number of dataset: 23


In [22]:
file_name = f'/home/hengjie/Desktop/{inf_file_name}_win{int_win}_sld{int_sld}.csv'

np.savetxt(file_name, combined_data, delimiter=',', header='phase,lead_matrix,lead_vector', comments='')
print(f'Data saved to {file_name}')

NameError: name 'int_win' is not defined

time taken for leading eigenvector: 232.10728291794658
rice bin size = 70
time taken: 3.610794077045284
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec matrix]: 0.8576918554651951
rice bin size = 34
time taken: 1.1163544160081074
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec]: 0.7440791157608969
rice bin size = 70
time taken: 3.5055408549960703
PDF evolution estimatation via histogram done!!!
average information length for [phase lock matrix]: 0.8576918554651951
time taken for leading eigenvector: 237.6350665370701
rice bin size = 70
time taken: 3.690295559936203
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec matrix]: 0.8974405773279964
rice bin size = 34
time taken: 1.127992736059241
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec]: 0.769752650077885
rice bin size = 70
time taken: 3.6394181300420314
PDF e

In [18]:
#interested eeg file
int_eeg_file = con_eeg_file[0:10]
int_file_name = 'con_eeg_file'

all_data_phase = []
all_data_leadmax = []
all_data_leadvec = []

def info_cal(file):
    f = file #file name
    #step 1: read the data using [mne.io.read_raw_eeglab()]
    temp_data = mne.io.read_raw_eeglab(f, preload=True)

    ###################################################################################
    #step 2: phase lock matrix calculation
    from scipy.signal import hilbert

    temp_hilbert_data = hilbert(temp_data[temp_data.ch_names][0]) #hilbert transform the all channels
    #temp_hilbert_time = temp_data[temp_data.ch_names][1] #keep the values for the [time]
    temp_hilbert_angle = np.angle(temp_hilbert_data, deg=False) #finding the [angle in radians] for all the signals

    hilbert_data = mne.io.RawArray(temp_hilbert_data, temp_data.info)
    temp_hilbert_angle = np.angle(hilbert_data[hilbert_data.ch_names][0], deg=False)
    hilbert_angle = mne.io.RawArray(temp_hilbert_angle, temp_data.info)

    matrix_hilbert_angle, chnl_name = phase_lock_matrix(hilbert_angle)
    ###################################################################################

    ###################################################################################
    #step 3: leading eigenvector of the phase lock matrix
    high_eigvec = lead_eigvec_cal(matrix_hilbert_angle)
    ###################################################################################



    #window sliding the data for the grouping. 
    int_win = 250 #window size
    int_sld = 100 #sliding of the window
    temp_data_timesld = temp_data.times[::int_sld] #window slided time information

    ###################################################################################
    #step 4: Information rate calculation for the [upper triangle matrix] of [leading eigenvector matrix]
    #calculate the [square matrix of the leading eigenvector]
    lead_matrix = np.einsum('ijk, imn -> ijm', high_eigvec, high_eigvec)

    #getting the upper triangle information 
    loc_lead_matrix_up = np.triu_indices(n=lead_matrix.shape[1], k=1)
    lead_matrix_up = lead_matrix[:, loc_lead_matrix_up[0], loc_lead_matrix_up[1]]
    lead_matrix_up = np.expand_dims(lead_matrix_up, axis=2)

    temp_data_sld = win_sld(lead_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his, temp_data_axs = hist_est(temp_data_sld, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate = np.sum(np.diff((temp_data_his**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs[1]-temp_data_axs[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen = np.cumsum((temp_inforate)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen = temp_infolen[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec matrix]: {temp_aveinfolen}')
    
    #save the data
    all_data_leadmax.append(temp_aveinfolen) 
    ###################################################################################

    ###################################################################################
    #step 5: Information rate calculation for the [leading eigenvector] 
    temp_data_sld_highvec = win_sld(high_eigvec, window=int_win, slide=int_sld)

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_highvec.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_highvec.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_highvec.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_highvec, temp_data_axs_highvec = hist_est(temp_data_sld_highvec, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_highvec = np.sum(np.diff((temp_data_his_highvec**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_highvec[1]-temp_data_axs_highvec[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_highvec = np.cumsum((temp_inforate_highvec)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_highvec = temp_infolen_highvec[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec]: {temp_aveinfolen_highvec}')
    
    #save the data 
    all_data_leadvec.append(temp_aveinfolen_highvec)
    ###################################################################################

    ###################################################################################
    #step 6: Information rate calculation for [phase lock matrix]
    #calculate the [square matrix of the leading eigenvector]
    phase_matrix = matrix_hilbert_angle.T

    #getting the upper triangle information 
    loc_phase_matrix_up = np.triu_indices(n=phase_matrix.shape[1], k=1)
    phase_matrix_up = lead_matrix[:, loc_phase_matrix_up[0], loc_phase_matrix_up[1]]
    phase_matrix_up = np.expand_dims(phase_matrix_up, axis=2)

    temp_data_sld_phase = win_sld(phase_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_phase.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_phase.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_phase.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_phase, temp_data_axs_phase = hist_est(temp_data_sld_phase, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_phase = np.sum(np.diff((temp_data_his_phase**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_phase[1]-temp_data_axs_phase[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_phase = np.cumsum((temp_inforate_phase)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_phase = temp_infolen_phase[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [phase lock matrix]: {temp_aveinfolen_phase}')
    
    #save the data 
    all_data_phase.append(temp_aveinfolen_phase)
    ###################################################################################
    return all_data_phase, all_data_leadmax, all_data_leadvec

all_data_phase, all_data_leadmax, all_data_leadvec = zip(*Parallel(n_jobs=-1)(delayed(info_cal)(f) for f in tqdm(int_eeg_file)))

#saving the data in .csv
all_data_phase = np.array(all_data_phase)
all_data_leadmax = np.array(all_data_leadmax)
all_data_leadvec = np.array(all_data_leadvec)

combined_data = np.vstack((all_data_phase, all_data_leadmax, all_data_leadvec)).T

file_name = f'/home/hengjie/Desktop/{int_file_name}_win{temp_win}_sld{temp_sld}.csv'

np.savetxt(file_name, combined_data, delimiter=',', header='phase,lead_matrix,lead_vector', comments='')
print(f'Data saved to {file_name}')


'''
#interested eeg file
int_eeg_file = alz_eeg_file
int_file_name = 'alz_eeg_file'

all_data_phase = []
all_data_leadmax = []
all_data_leadvec = []

for f in tqdm(int_eeg_file):
    #step 1: read the data using [mne.io.read_raw_eeglab()]
    temp_data = mne.io.read_raw_eeglab(f, preload=True)

    ###################################################################################
    #step 2: phase lock matrix calculation
    from scipy.signal import hilbert

    temp_hilbert_data = hilbert(temp_data[temp_data.ch_names][0]) #hilbert transform the all channels
    #temp_hilbert_time = temp_data[temp_data.ch_names][1] #keep the values for the [time]
    temp_hilbert_angle = np.angle(temp_hilbert_data, deg=False) #finding the [angle in radians] for all the signals

    hilbert_data = mne.io.RawArray(temp_hilbert_data, temp_data.info)
    temp_hilbert_angle = np.angle(hilbert_data[hilbert_data.ch_names][0], deg=False)
    hilbert_angle = mne.io.RawArray(temp_hilbert_angle, temp_data.info)

    matrix_hilbert_angle, chnl_name = phase_lock_matrix(hilbert_angle)
    ###################################################################################

    ###################################################################################
    #step 3: leading eigenvector of the phase lock matrix
    high_eigvec = lead_eigvec_cal(matrix_hilbert_angle)
    ###################################################################################



    #window sliding the data for the grouping. 
    int_win = 250 #window size
    int_sld = 100 #sliding of the window
    temp_data_timesld = temp_data.times[::int_sld] #window slided time information

    ###################################################################################
    #step 4: Information rate calculation for the [upper triangle matrix] of [leading eigenvector matrix]
    #calculate the [square matrix of the leading eigenvector]
    lead_matrix = np.einsum('ijk, imn -> ijm', high_eigvec, high_eigvec)

    #getting the upper triangle information 
    loc_lead_matrix_up = np.triu_indices(n=lead_matrix.shape[1], k=1)
    lead_matrix_up = lead_matrix[:, loc_lead_matrix_up[0], loc_lead_matrix_up[1]]
    lead_matrix_up = np.expand_dims(lead_matrix_up, axis=2)

    temp_data_sld = win_sld(lead_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his, temp_data_axs = hist_est(temp_data_sld, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate = np.sum(np.diff((temp_data_his**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs[1]-temp_data_axs[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen = np.cumsum((temp_inforate)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen = temp_infolen[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec matrix]: {temp_aveinfolen}')
    
    #save the data
    all_data_leadmax.append(temp_aveinfolen) 
    ###################################################################################

    ###################################################################################
    #step 5: Information rate calculation for the [leading eigenvector] 
    temp_data_sld_highvec = win_sld(high_eigvec, window=int_win, slide=int_sld)

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_highvec.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_highvec.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_highvec.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_highvec, temp_data_axs_highvec = hist_est(temp_data_sld_highvec, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_highvec = np.sum(np.diff((temp_data_his_highvec**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_highvec[1]-temp_data_axs_highvec[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_highvec = np.cumsum((temp_inforate_highvec)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_highvec = temp_infolen_highvec[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec]: {temp_aveinfolen_highvec}')
    
    #save the data 
    all_data_leadvec.append(temp_aveinfolen_highvec)
    ###################################################################################

    ###################################################################################
    #step 6: Information rate calculation for [phase lock matrix]
    #calculate the [square matrix of the leading eigenvector]
    phase_matrix = matrix_hilbert_angle.T

    #getting the upper triangle information 
    loc_phase_matrix_up = np.triu_indices(n=phase_matrix.shape[1], k=1)
    phase_matrix_up = lead_matrix[:, loc_phase_matrix_up[0], loc_phase_matrix_up[1]]
    phase_matrix_up = np.expand_dims(phase_matrix_up, axis=2)

    temp_data_sld_phase = win_sld(phase_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_phase.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_phase.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_phase.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_phase, temp_data_axs_phase = hist_est(temp_data_sld_phase, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_phase = np.sum(np.diff((temp_data_his_phase**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_phase[1]-temp_data_axs_phase[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_phase = np.cumsum((temp_inforate_phase)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_phase = temp_infolen_phase[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [phase lock matrix]: {temp_aveinfolen_phase}')
    
    #save the data 
    all_data_phase.append(temp_aveinfolen_phase)
    ###################################################################################
    
#saving the data in .csv
all_data_phase = np.array(all_data_phase)
all_data_leadmax = np.array(all_data_leadmax)
all_data_leadvec = np.array(all_data_leadvec)

combined_data = np.vstack((all_data_phase, all_data_leadmax, all_data_leadvec)).T

file_name = f'/home/hengjie/Desktop/{int_file_name}_win{int_win}_sld{int_sld}.csv'

np.savetxt(file_name, combined_data, delimiter=',', header='phase,lead_matrix,lead_vector', comments='')
print(f'Data saved to {file_name}')
'''

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 9349.76it/s]
/tmp/ipykernel_90477/1197340617.py:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_90477/1197340617.py:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_90477/1197340617.py:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_90477/1197340617.py:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/ipykernel_90477/1197340617.py:12: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of fil

Creating RawArray with complex128 data, n_channels=19, n_times=425040
    Range : 0 ... 425039 =      0.000 ...   850.078 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=425040
    Range : 0 ... 425039 =      0.000 ...   850.078 secs
Ready.
Creating RawArray with complex128 data, n_channels=19, n_times=440800
    Range : 0 ... 440799 =      0.000 ...   881.598 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=440800
    Range : 0 ... 440799 =      0.000 ...   881.598 secs
Ready.
Creating RawArray with complex128 data, n_channels=19, n_times=425820
    Range : 0 ... 425819 =      0.000 ...   851.638 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=425820
    Range : 0 ... 425819 =      0.000 ...   851.638 secs
Ready.
Creating RawArray with complex128 data, n_channels=19, n_times=376520
    Range : 0 ... 376519 =      0.000 ...   753.038 secs
Ready.
Creating RawArray with complex128 data, n_channels=19, n_times=388550
 

NameError: name 'int_file_name' is not defined

In [37]:
'''
#interested eeg file
int_eeg_file = con_eeg_file[0:10]
inf_file_name = 'con_eeg_file'

all_data_phase = []
all_data_leadmax = []
all_data_leadvec = []

def info_cal(file, temp_win, temp_sld):
    f = file #file name
    
    #step 1: read the data using [mne.io.read_raw_eeglab()]
    temp_data = mne.io.read_raw_eeglab(f, preload=True)

    ###################################################################################
    #step 2: phase lock matrix calculation
    from scipy.signal import hilbert

    temp_hilbert_data = hilbert(temp_data[temp_data.ch_names][0]) #hilbert transform the all channels
    #temp_hilbert_time = temp_data[temp_data.ch_names][1] #keep the values for the [time]
    temp_hilbert_angle = np.angle(temp_hilbert_data, deg=False) #finding the [angle in radians] for all the signals

    hilbert_data = mne.io.RawArray(temp_hilbert_data, temp_data.info)
    temp_hilbert_angle = np.angle(hilbert_data[hilbert_data.ch_names][0], deg=False)
    hilbert_angle = mne.io.RawArray(temp_hilbert_angle, temp_data.info)

    matrix_hilbert_angle, chnl_name = phase_lock_matrix(hilbert_angle)
    ###################################################################################

    ###################################################################################
    #step 3: leading eigenvector of the phase lock matrix
    high_eigvec = lead_eigvec_cal(matrix_hilbert_angle)
    ###################################################################################



    #window sliding the data for the grouping.
    int_win = temp_win #window size
    int_sld = temp_sld #sliding of the window
    temp_data_timesld = temp_data.times[::int_sld] #window slided time information

    ###################################################################################
    #step 4: Information rate calculation for the [upper triangle matrix] of [leading eigenvector matrix]
    #calculate the [square matrix of the leading eigenvector]
    lead_matrix = np.einsum('ijk, imn -> ijm', high_eigvec, high_eigvec)

    #getting the upper triangle information
    loc_lead_matrix_up = np.triu_indices(n=lead_matrix.shape[1], k=1)
    lead_matrix_up = lead_matrix[:, loc_lead_matrix_up[0], loc_lead_matrix_up[1]]
    lead_matrix_up = np.expand_dims(lead_matrix_up, axis=2)

    temp_data_sld = win_sld(lead_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his, temp_data_axs = hist_est(temp_data_sld, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate = np.sum(np.diff((temp_data_his**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs[1]-temp_data_axs[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen = np.cumsum((temp_inforate)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen = temp_infolen[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec matrix]: {temp_aveinfolen}')

    #save the data
    all_data_leadmax.append(temp_aveinfolen)
    ###################################################################################

    ###################################################################################
    #step 5: Information rate calculation for the [leading eigenvector]
    temp_data_sld_highvec = win_sld(high_eigvec, window=int_win, slide=int_sld)

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_highvec.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_highvec.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_highvec.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_highvec, temp_data_axs_highvec = hist_est(temp_data_sld_highvec, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_highvec = np.sum(np.diff((temp_data_his_highvec**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_highvec[1]-temp_data_axs_highvec[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_highvec = np.cumsum((temp_inforate_highvec)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_highvec = temp_infolen_highvec[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec]: {temp_aveinfolen_highvec}')

    #save the data
    all_data_leadvec.append(temp_aveinfolen_highvec)
    ###################################################################################

    ###################################################################################
    #step 6: Information rate calculation for [phase lock matrix]
    #calculate the [square matrix of the leading eigenvector]
    phase_matrix = matrix_hilbert_angle.T

    #getting the upper triangle information
    loc_phase_matrix_up = np.triu_indices(n=phase_matrix.shape[1], k=1)
    phase_matrix_up = lead_matrix[:, loc_phase_matrix_up[0], loc_phase_matrix_up[1]]
    phase_matrix_up = np.expand_dims(phase_matrix_up, axis=2)

    temp_data_sld_phase = win_sld(phase_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_phase.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_phase.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_phase.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_phase, temp_data_axs_phase = hist_est(temp_data_sld_phase, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_phase = np.sum(np.diff((temp_data_his_phase**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_phase[1]-temp_data_axs_phase[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_phase = np.cumsum((temp_inforate_phase)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_phase = temp_infolen_phase[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [phase lock matrix]: {temp_aveinfolen_phase}')

    #save the data
    all_data_phase.append(temp_aveinfolen_phase)
    ###################################################################################
    return all_data_phase, all_data_leadmax, all_data_leadvec

start = timer()
all_data_phase, all_data_leadmax, all_data_leadvec = zip(*Parallel(n_jobs=-1)(delayed(info_cal)(f, temp_win=250, temp_sld=100) for f in tqdm(int_eeg_file)))
print(f'time taken for all calculation: {timer() - start}')

#saving the data in .csv
all_data_phase = np.array(all_data_phase)
all_data_leadmax = np.array(all_data_leadmax)
all_data_leadvec = np.array(all_data_leadvec)

combined_data = np.vstack((all_data_phase, all_data_leadmax, all_data_leadvec)).T

file_name = f'C:/Users/choongh2/Desktop/{int_file_name}_win{temp_win}_sld{temp_sld}.csv'

np.savetxt(file_name, combined_data, delimiter=',', header='phase,lead_matrix,lead_vector', comments='')
print(f'Data saved to {file_name}')

'''

#interested eeg file
int_eeg_file = dem_eeg_file
int_file_name = 'dem_eeg_file'

all_data_phase = []
all_data_leadmax = []
all_data_leadvec = []

for f in tqdm(int_eeg_file):
    #step 1: read the data using [mne.io.read_raw_eeglab()]
    temp_data = mne.io.read_raw_eeglab(f, preload=True)

    ###################################################################################
    #step 2: phase lock matrix calculation
    from scipy.signal import hilbert

    temp_hilbert_data = hilbert(temp_data[temp_data.ch_names][0]) #hilbert transform the all channels
    #temp_hilbert_time = temp_data[temp_data.ch_names][1] #keep the values for the [time]
    temp_hilbert_angle = np.angle(temp_hilbert_data, deg=False) #finding the [angle in radians] for all the signals

    hilbert_data = mne.io.RawArray(temp_hilbert_data, temp_data.info)
    temp_hilbert_angle = np.angle(hilbert_data[hilbert_data.ch_names][0], deg=False)
    hilbert_angle = mne.io.RawArray(temp_hilbert_angle, temp_data.info)

    matrix_hilbert_angle, chnl_name = phase_lock_matrix(hilbert_angle)
    ###################################################################################

    ###################################################################################
    #step 3: leading eigenvector of the phase lock matrix
    high_eigvec = lead_eigvec_cal(matrix_hilbert_angle)
    ###################################################################################



    #window sliding the data for the grouping.
    int_win = 2 #window size
    int_sld = 1 #sliding of the window
    temp_data_timesld = temp_data.times[::int_sld] #window slided time information

    ###################################################################################
    #step 4: Information rate calculation for the [upper triangle matrix] of [leading eigenvector matrix]
    #calculate the [square matrix of the leading eigenvector]
    lead_matrix = np.einsum('ijk, imn -> ijm', high_eigvec, high_eigvec)

    #getting the upper triangle information
    loc_lead_matrix_up = np.triu_indices(n=lead_matrix.shape[1], k=1)
    lead_matrix_up = lead_matrix[:, loc_lead_matrix_up[0], loc_lead_matrix_up[1]]
    lead_matrix_up = np.expand_dims(lead_matrix_up, axis=2)

    temp_data_sld = win_sld(lead_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his, temp_data_axs = hist_est(temp_data_sld, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate = np.sum(np.diff((temp_data_his**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs[1]-temp_data_axs[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen = np.cumsum((temp_inforate)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen = temp_infolen[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec matrix]: {temp_aveinfolen}')

    #save the data
    all_data_leadmax.append(temp_aveinfolen)
    ###################################################################################

    ###################################################################################
    #step 5: Information rate calculation for the [leading eigenvector]
    temp_data_sld_highvec = win_sld(high_eigvec, window=int_win, slide=int_sld)

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_highvec.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_highvec.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_highvec.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_highvec, temp_data_axs_highvec = hist_est(temp_data_sld_highvec, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_highvec = np.sum(np.diff((temp_data_his_highvec**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_highvec[1]-temp_data_axs_highvec[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_highvec = np.cumsum((temp_inforate_highvec)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_highvec = temp_infolen_highvec[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [lead eigvec]: {temp_aveinfolen_highvec}')

    #save the data
    all_data_leadvec.append(temp_aveinfolen_highvec)
    ###################################################################################

    ###################################################################################
    #step 6: Information rate calculation for [phase lock matrix]
    #calculate the [square matrix of the leading eigenvector]
    phase_matrix = matrix_hilbert_angle.T

    #getting the upper triangle information
    loc_phase_matrix_up = np.triu_indices(n=phase_matrix.shape[1], k=1)
    phase_matrix_up = lead_matrix[:, loc_phase_matrix_up[0], loc_phase_matrix_up[1]]
    phase_matrix_up = np.expand_dims(phase_matrix_up, axis=2)

    temp_data_sld_phase = win_sld(phase_matrix_up, window=int_win, slide=int_sld) #window slided data

    #if the [temp_data_timesld] higher/not equal to [temp_data_sld]'s time --> pickup to the point they are the same size.
    #This is needed because the [temp_data_sld] may not include the last few data due to the [window size].
    if temp_data_timesld.shape[0] != temp_data_sld_phase.shape[0]:
        temp_data_timesld = temp_data_timesld[:temp_data_sld_phase.shape[0]]

    #estimate the [evolution of histogram] using the [numpy.histogram] and [numpy.apply_along_axis]
    rice_size = int(np.ceil(2 * (temp_data_sld_phase.shape[1])**(1/3))) #rice rule to estimate the bins size.
    print(f'rice bin size = {rice_size}')

    temp_data_his_phase, temp_data_axs_phase = hist_est(temp_data_sld_phase, bins_size=rice_size, )

    #calculate the [information rate]
    temp_inforate_phase = np.sum(np.diff((temp_data_his_phase**(1/2)), axis=0)**(2), axis=1) * ((temp_data_axs_phase[1]-temp_data_axs_phase[0])/((temp_data_timesld[1]-temp_data_timesld[0])**2))

    #calculate the [information length]
    temp_infolen_phase = np.cumsum((temp_inforate_phase)**(1/2)) * (temp_data_timesld[1]-temp_data_timesld[0])

    #calculate the [average of information length]
    temp_aveinfolen_phase = temp_infolen_phase[-1]/temp_data_timesld[-1]

    #the [average information length] is defined as ['total' information length](final information length) divided by [total time]
    print(f'average information length for [phase lock matrix]: {temp_aveinfolen_phase}')

    #save the data
    all_data_phase.append(temp_aveinfolen_phase)
    ###################################################################################

#saving the data in .csv
all_data_phase = np.array(all_data_phase)
all_data_leadmax = np.array(all_data_leadmax)
all_data_leadvec = np.array(all_data_leadvec)

combined_data = np.vstack((all_data_phase, all_data_leadmax, all_data_leadvec)).T

file_name = f'/home/hengjie/Desktop/{int_file_name}_win{int_win}_sld{int_sld}.csv'

np.savetxt(file_name, combined_data, delimiter=',', header='phase,lead_matrix,lead_vector', comments='')
print(f'Data saved to {file_name}')


  0%|                                                                                            | 0/23 [00:00<?, ?it/s]/tmp/ipykernel_90477/4062675380.py:181: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  temp_data = mne.io.read_raw_eeglab(f, preload=True)


Creating RawArray with complex128 data, n_channels=19, n_times=274880
    Range : 0 ... 274879 =      0.000 ...   549.758 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=274880
    Range : 0 ... 274879 =      0.000 ...   549.758 secs
Ready.
time taken for leading eigenvector: 12.501944673946127
rice bin size = 14
time taken: 48.49105157610029
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec matrix]: 48.294251238630906
rice bin size = 7
time taken: 30.586851069005206
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec]: 32.64836796779345
rice bin size = 14


  4%|███▌                                                                               | 1/23 [02:26<53:50, 146.86s/it]

time taken: 48.45425082498696
PDF evolution estimatation via histogram done!!!
average information length for [phase lock matrix]: 48.294251238630906
Creating RawArray with complex128 data, n_channels=19, n_times=321550
    Range : 0 ... 321549 =      0.000 ...   643.098 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=321550
    Range : 0 ... 321549 =      0.000 ...   643.098 secs
Ready.
time taken for leading eigenvector: 13.45304774097167
rice bin size = 14
time taken: 57.36808636004571
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec matrix]: 106.06011853556922
rice bin size = 7
time taken: 35.17946835805196
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec]: 91.58123375000321
rice bin size = 14


  9%|███████▏                                                                           | 2/23 [05:16<56:11, 160.53s/it]

...

time taken: 53.18372966896277
PDF evolution estimatation via histogram done!!!
average information length for [phase lock matrix]: 35.422010513153296


/tmp/ipykernel_90477/4062675380.py:181: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  temp_data = mne.io.read_raw_eeglab(f, preload=True)


Creating RawArray with complex128 data, n_channels=19, n_times=392350
    Range : 0 ... 392349 =      0.000 ...   784.698 secs
Ready.
Creating RawArray with float64 data, n_channels=19, n_times=392350
    Range : 0 ... 392349 =      0.000 ...   784.698 secs
Ready.
time taken for leading eigenvector: 12.786182718002237
rice bin size = 14
time taken: 68.40073736500926
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec matrix]: 46.29401062082746
rice bin size = 7
time taken: 42.80530079896562
PDF evolution estimatation via histogram done!!!
average information length for [lead eigvec]: 27.633427334354707
rice bin size = 14


100%|████████████████████████████████████████████████████████████████████████████████| 23/23 [1:13:47<00:00, 192.48s/it]

time taken: 68.7361723780632
PDF evolution estimatation via histogram done!!!
average information length for [phase lock matrix]: 46.29401062082746
Data saved to /home/hengjie/Desktop/dem_eeg_file_win2_sld1.csv


# Interpret the calculated data

In [50]:
read_win = 2
read_sld = 1
read_file = f'dem_eeg_file_win{read_win}_sld{read_sld}'

current_data = pd.read_csv(f'/home/hengjie/Desktop/{read_file}.csv')

In [51]:
current_data

,phase,lead_matrix,lead_vector
0,48.294251,48.294251,32.648368
1,106.060119,106.060119,91.581234
2,29.659338,29.659338,16.594520
3,64.406298,64.406298,50.872753
4,33.911557,33.911557,17.858036
5,40.140484,40.140484,20.738244
6,45.549014,45.549014,31.544385
7,26.853789,26.853789,14.789736
8,35.231857,35.231857,21.486486
9,68.435758,68.435758,44.724364


In [54]:
temp_data_timesld

array([0.00000e+00, 2.00000e-03, 4.00000e-03, ..., 7.84692e+02,
       7.84694e+02, 7.84696e+02])

In [55]:
1/500

0.002

In [52]:
read_phase_ave = np.mean(current_data['phase'])
read_phase_std = np.std(current_data['phase'])

read_leadmax_ave = np.mean(current_data['lead_matrix'])
read_leadmax_std = np.std(current_data['lead_matrix'])

read_leadvec_ave = np.mean(current_data['lead_vector'])
read_leadvec_std = np.std(current_data['lead_vector'])

print(f'phase: {read_phase_ave} +/- {read_phase_std}')
print(f'lead matrix: {read_leadmax_ave} +/- {read_leadmax_std}')
print(f'lead vector: {read_leadvec_ave} +/- {read_leadvec_std}')

phase: 48.24500062395426 +/- 20.672929023689814
lead matrix: 48.24500062395426 +/- 20.672929023689814
lead vector: 33.05314820217774 +/- 21.95039459357971


In [ ]:
#window=2, slide=1
#control --> 29 patients
phase: 43.93668206634839 +/- 8.518433661506235
lead matrix: 43.93668206634839 +/- 8.518433661506235
lead vector: 27.514436031935258 +/- 7.888432418974941

#alzheimer --> 36 patients
phase: 43.48364092475495 +/- 11.480609701076387
lead matrix: 43.48364092475495 +/- 11.480609701076387
lead vector: 27.935481741122707 +/- 10.951475199697066

#dementia --> 23 patients
phase: 43.48364092475495 +/- 11.480609701076387
lead matrix: 43.48364092475495 +/- 11.480609701076387
lead vector: 27.935481741122707 +/- 10.951475199697066

In [ ]:
#control 
phase: 0.20108773780054348 +/- 0.009071454147605395
lead matrix: 0.20108773780054348 +/- 0.009071454147605395
lead vector: 0.1737642133404058 +/- 0.011334504651834868

#alzheimer
phase: 0.20609927213068652 +/- 0.011583803628787387
lead matrix: 0.20609927213068652 +/- 0.011583803628787387
lead vector: 0.17740273950470187 +/- 0.012742944635028534

#dementia
phase: 0.1931605858115161 +/- 0.015099754945341966
lead matrix: 0.1931605858115161 +/- 0.015099754945341966
lead vector: 0.16828455610431495 +/- 0.01442185252451492
